# 2. Feature Engineering and Modifying Data For Prediction Model

In [1]:
import pandas as pd
import numpy as np

### Compare Train Data Set and Test Data Set

In [27]:
test = pd.read_csv('test.csv', encoding = 'utf-8')
train = pd.read_csv('train.csv', encoding = 'utf-8')
songs = pd.read_csv('songs.csv', encoding = 'utf-8')
members = pd.read_csv('members.csv', encoding = 'utf-8')

In [3]:
def compare_test_train(col):
    set_train = set(train[col])
    set_test = set(test[col])
    i = len(set_test - set_train -set([None]))
    print('number of test set  %s, not in train set' % col, ':', i)
    
def compare_train_test(col):
    set_train = set(train[col])
    set_test = set(test[col])
    i = len(set_train - set_test -set([None]))
    print('number of train set  %s, not in test set' % col, ':', i)

In [37]:
data_sum = [test, train]
for df in data_sum:
    df1 = pd.merge(df, songs, on='song_id')
    df2 = pd.merge(df1, members, on='msno')
    del df2['language']
    if len(df2) > 7000000:
        train = df2
    else:
        test = df2


In [34]:
test.columns

Index(['id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'song_length', 'genre_ids', 'artist_name', 'composer',
       'lyricist', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'expiration_date'],
      dtype='object')

In [43]:
train = train.fillna('no_value')
test = test.fillna('no_value')

In [46]:
category_var = ['source_system_tab', 'source_screen_name', 'source_type', 'city', 'gender', 'registered_via']

for var in category_var:
    compare_train_test(var)
for var in category_var:
    compare_test_train(var)
    

number of train set  source_system_tab, not in test set : 0
number of train set  source_screen_name, not in test set : 0
number of train set  source_type, not in test set : 0
number of train set  city, not in test set : 0
number of train set  gender, not in test set : 0
number of train set  registered_via, not in test set : 0
number of test set  source_system_tab, not in train set : 0
number of test set  source_screen_name, not in train set : 2
number of test set  source_type, not in train set : 0
number of test set  city, not in train set : 0
number of test set  gender, not in train set : 0
number of test set  registered_via, not in train set : 1


In [47]:
SCN = set(test['source_screen_name']) - set(train['source_screen_name'])
RV = set(test['registered_via']) - set(train['registered_via'])
print('Source screen name not in train set : ' , SCN)
print('registered_via name not in train set : ' ,RV)

Source screen name not in train set :  {'People local', 'People global'}
registered_via name not in train set :  {16}


In [48]:
lis = ['source_screen_name', 'registered_via']
for x in lis:
    if 'no_value' not in  set(train[x]):
        print(x)

registered_via


In [49]:
test.groupby(['registered_via']).size()

registered_via
3     418944
4     274347
7     901190
9     955103
13      7179
16         2
dtype: int64

In [50]:
lis = {'People global', 'People local'}

for x in lis:
    test['source_screen_name'] = test['source_screen_name'].replace(x, 'no_value')

test = test[test['registered_via'] != 16]

### Feature Engineering

In [51]:
def msno_count(df):
    df1 = df.groupby('msno').size()
    df1= df1.reset_index()
    df1.columns = ['msno', '#listened']
    return df1

# song_id, song_length
def mean_count_var(df, var):
    df1 = df[[var, 'msno']].groupby([var]).agg(['count'])
    df1 = df1.reset_index()
    df1.columns = [var, 'count']
    return df1

# genre_ids, lyricist, composer, artist_name
def player_preference(df, var, preference_by):
    df1 = df[['msno', var]].groupby(['msno', var]).size()
    df1 = df1.reset_index()
    df1.columns = ['msno', var, preference_by ]
    return df1
    #df1 = pd.merge(train, df, on = ['msno', var], how='left')

#몇명의 작곡가가 만들었나
#genre_ids, composer
def count_vals(x):
    if type(x) != str:
        return 1
    else:
        return 1 + x.count('|')


In [52]:
data_sum = [test, train]

In [ ]:
for df in data_sum:
    df1 = pd.merge(df, msno_count(df), on = 'msno')
    df2 = pd.merge(df, mean_count_var(df, 'song_id'), on = 'song_id')
    df3 = pd.merge(df, mean_count_var(df, 'song_length'), on = 'song_length')
    if len(df) > 7000000:
        train = df3
    else:
        test = df3

In [ ]:
for df in data_sum:
    df1 = pd.merge(df,  player_preference(df, 'genre_ids', 'by_genre'), on = ['msno', 'genre_ids'], how = 'left')
    df2 = pd.merge(df1,  player_preference(df, 'lyricist', 'by_lyricist'), on = ['msno', 'lyricist'], how = 'left')
    df3 = pd.merge(df2,  player_preference(df, 'composer', 'by_composer'), on = ['msno', 'composer'], how = 'left')
    df4 = pd.merge(df3,  player_preference(df, 'artist_name', 'by_artist'), on = ['msno', 'artist_name'], how = 'left')
    if len(df) > 7000000:
        train = df4
    else:
        test = df4

In [ ]:
train['number_of_genre'] =  train['genre_ids'].apply(count_vals)
train['number_of_composer'] =  train['composer'].apply(count_vals)

test['number_of_genre'] =  test['genre_ids'].apply(count_vals)
test['number_of_composer'] =  test['composer'].apply(count_vals)

In [ ]:
test = test[test['number_of_composer'] < 30]
test = test[test['bd'] > -1]
test = test[test['bd'] < 70]

train = train[train['number_of_composer'] < 30]
train = train[train['bd'] > -1]
train = train[train['bd'] < 70]

In [ ]:
source_system_tab_test = pd.get_dummies(test['source_system_tab'], drop_first = True)
source_screen_name_test = pd.get_dummies(test['source_screen_name'], drop_first = True)
source_type_test = pd.get_dummies(test['source_type'], drop_first = True)
city_test = pd.get_dummies(test['city'], drop_first = True)
gendera_test = pd.get_dummies(test['gender'], drop_first = True)
registered_test = pd.get_dummies(test['registered_via'], drop_first = True)

In [ ]:
test = test[['song_length', 'bd', '#listened', 'count_x', 'count_y', 'by_genre', 'by_lyricist',
             'by_composer', 'by_artist', 'number_of_genre', 'number_of_composer']]

In [ ]:
frame = [test, source_system_tab_test , source_screen_name_test, source_type_test, 
         city_test, gender_test, registered_via_test]
test = pd.concat(frame, axis = 1)

In [ ]:
test.to_csv('test_final', encoding = 'utf-8', index=False)

In [ ]:
i = 0
for x in range(500000, 7000001, 500000):
    i += 1
    x1 = x-500000
    df = train.iloc[x1:x, :]
    df.to_csv('train%s' %i, encoding = 'utf-8', index=False)
    
df = train.iloc[7000000:,:]
df.to_csv('train15', encoding = 'utf-8', index = False)

In [ ]:
def to_cat(i):
    df = pd.read_csv('train%s' %i, encoding = 'utf-8')
    source_system_tab = pd.get_dummies(df['source_system_tab'], drop_first = True)
    source_screen_name = pd.get_dummies(df['source_screen_name'], drop_first = True)
    source_type = pd.get_dummies(df['source_type'], drop_first = True)
    city = pd.get_dummies(df['city'], drop_first = True)
    gender = pd.get_dummies(df['gender'], drop_first = True)
    registered_via = pd.get_dummies(df['registered_via'], drop_first = True)
    
    df = df[['target','song_length', 'bd', '#listened', 'count', 'count1', 'by_genre',
       'by_lyricist', 'by_composer', 'by_artist', 'number_of_genre',
       'number_of_composer']]
    frame = [df, source_system_tab , source_screen_name, source_type, city, gender, registered_via]
    df1 = pd.concat(frame, axis = 1)
    df1.to_csv('train_final%s' %i, encoding = 'utf-8', index = False)
    
    

In [ ]:
for x in range(1, 16):
    to_cat(x)

## Create Train Data Set and Evaluation(test) Data Set

In [ ]:
train = pd.read_csv('train_final1', encoding = 'utf-8')
for i in range(2,16):
    df = pd.read_csv('train_final%s' %i, encoding = 'utf-8')
    train = pd.concat([train, df])

In [ ]:
train.columns

In [ ]:
y = train['target']
del train['target']
X = train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

np.save("X_train", X_train)
np.save("X_test", X_test)
np.save("y_train", y_train)
np.save("y_test", y_test)

In [ ]:
test = pd.read_csv('test_final')
test = test.reindex_axis(sorted(test.columns), axis=1)
test = np.array(test)
np.save('X_test_input', test)